In [0]:
empDF1 = spark.read.format("csv").options(header= True, inferSchema= True, sep= ",") \
    .load("/FileStore/tables/Employee_info.csv")

empDF1.show(20, False)
empDF1.printSchema()

+----+--------+----------------+
|Id  |Name    |Department      |
+----+--------+----------------+
|100 |Aman    |Data Engineering|
|200 |Saurabh |Data Engineering|
|300 |Mohit   |DavOps          |
|400 |Kashif  |DAvOps          |
|500 |Eniya   |DAvOps          |
|600 |Anand   |DAvOps          |
|700 |Murali  |Data Engineering|
|800 |Ramesh  |Null            |
|900 |Suresh  |Null            |
|1000|Himanshu|Null            |
+----+--------+----------------+

root
 |-- Id: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Department: string (nullable = true)



In [0]:
empDF2 = spark.read.format("csv").options(header= True, inferSchema= True, sep= ",") \
    .load("/FileStore/tables/Employee_info_1.csv")

empDF2.show(20, False)
empDF2.printSchema()

+----+-----------+--------+-----+
|Id  |Employee_id|City    |State|
+----+-----------+--------+-----+
|100 |2555       |Indore  |MP   |
|200 |2456       |Indore  |MP   |
|300 |3265       |Surat   |GJ   |
|400 |7896       |Banglore|KA   |
|500 |4562       |Banglore|KA   |
|600 |8524       |Banglore|KA   |
|700 |6666       |Banglore|KA   |
|800 |9853       |Banglore|KA   |
|900 |1594       |Null    |JK   |
|1000|7894       |Null    |JK   |
+----+-----------+--------+-----+

root
 |-- Id: integer (nullable = true)
 |-- Employee_id: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)



In [0]:
empDF = empDF1.join(empDF2, "Id", "fullouter").orderBy(empDF1.Id)

empDF.show()
empDF.printSchema()

+----+--------+----------------+-----------+--------+-----+
|  Id|    Name|      Department|Employee_id|    City|State|
+----+--------+----------------+-----------+--------+-----+
| 100|    Aman|Data Engineering|       2555|  Indore|   MP|
| 200| Saurabh|Data Engineering|       2456|  Indore|   MP|
| 300|   Mohit|          DavOps|       3265|   Surat|   GJ|
| 400|  Kashif|          DAvOps|       7896|Banglore|   KA|
| 500|   Eniya|          DAvOps|       4562|Banglore|   KA|
| 600|   Anand|          DAvOps|       8524|Banglore|   KA|
| 700|  Murali|Data Engineering|       6666|Banglore|   KA|
| 800|  Ramesh|            Null|       9853|Banglore|   KA|
| 900|  Suresh|            Null|       1594|    Null|   JK|
|1000|Himanshu|            Null|       7894|    Null|   JK|
+----+--------+----------------+-----------+--------+-----+

root
 |-- Id: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- Employee_id: integer (nullable = tru

In [0]:
from pyspark.sql.functions import col, count, when

CountNull = empDF.select([count(when(col(c).isNull(), c)).alias(c) for c in empDF.columns])
#display(CountNull)
CountNull.show(20, False)

+---+----+----------+-----------+----+-----+
|Id |Name|Department|Employee_id|City|State|
+---+----+----------+-----------+----+-----+
|0  |0   |0         |0          |0   |0    |
+---+----+----------+-----------+----+-----+



In [0]:
empDF_n = empDF.fillna({"Department": "No Dept Allocated", "City": "Shrinagar"})

empDF_n.show(20, False)

+----+--------+----------------+-----------+--------+-----+
|Id  |Name    |Department      |Employee_id|City    |State|
+----+--------+----------------+-----------+--------+-----+
|100 |Aman    |Data Engineering|2555       |Indore  |MP   |
|200 |Saurabh |Data Engineering|2456       |Indore  |MP   |
|300 |Mohit   |DavOps          |3265       |Surat   |GJ   |
|400 |Kashif  |DAvOps          |7896       |Banglore|KA   |
|500 |Eniya   |DAvOps          |4562       |Banglore|KA   |
|600 |Anand   |DAvOps          |8524       |Banglore|KA   |
|700 |Murali  |Data Engineering|6666       |Banglore|KA   |
|800 |Ramesh  |Null            |9853       |Banglore|KA   |
|900 |Suresh  |Null            |1594       |Null    |JK   |
|1000|Himanshu|Null            |7894       |Null    |JK   |
+----+--------+----------------+-----------+--------+-----+



In [0]:
from pyspark.sql.functions import regexp_replace

empDF_n = empDF.withColumn('Department', regexp_replace('Department', 'Null', 'Other Dept')) \
              .withColumn('City', regexp_replace('City', 'Null', 'Shrinagar'))

empDF_n.show(truncate= False)

+----+--------+----------------+-----------+---------+-----+
|Id  |Name    |Department      |Employee_id|City     |State|
+----+--------+----------------+-----------+---------+-----+
|100 |Aman    |Data Engineering|2555       |Indore   |MP   |
|200 |Saurabh |Data Engineering|2456       |Indore   |MP   |
|300 |Mohit   |DavOps          |3265       |Surat    |GJ   |
|400 |Kashif  |DAvOps          |7896       |Banglore |KA   |
|500 |Eniya   |DAvOps          |4562       |Banglore |KA   |
|600 |Anand   |DAvOps          |8524       |Banglore |KA   |
|700 |Murali  |Data Engineering|6666       |Banglore |KA   |
|800 |Ramesh  |Other Dept      |9853       |Banglore |KA   |
|900 |Suresh  |Other Dept      |1594       |Shrinagar|JK   |
|1000|Himanshu|Other Dept      |7894       |Shrinagar|JK   |
+----+--------+----------------+-----------+---------+-----+



In [0]:
from pyspark.sql.functions import countDistinct

dept_count = empDF_n.select().where(empDF_n.Department=='Other Dept').count()
print("Department Null count: " + str(dept_count))
city_count = empDF_n.select().where(empDF_n.City=='Shrinagar').count()
print("City Null count: " + str(city_count))

Null_count = empDF_n.select().where((empDF_n.Department=='Null') | (empDF_n.City=='Null')).count()
print("Count of Null Values: " + str(Null_count))

Department Null count: 3
City Null count: 2
Count of Null Values: 0
